# OPTIONAL:  Define Wells

In [1]:
# Import Libraries
%run /Users/u4eeevmq/Documents/Python/HyporheicFloPy/VQuintana/common_imports.py

# Retrieve stored variables
%store -r md6_exe_path
%store -r md7_exe_path
%store -r sim_name
%store -r workspace
%store -r figs_path
%store -r gwf_name
%store -r prt_name
%store -r mp7_name
%store -r gwf_ws
%store -r prt_ws
%store -r mp7_ws
%store -r headfile
%store -r head_filerecord
%store -r headfile_prt
%store -r budgetfile
%store -r budget_filerecord
%store -r budgetfile_prt
%store -r budget_filerecord_prt
%store -r trackfile_prt
%store -r trackhdrfile_prt
%store -r trackcsvfile_prt
%store -r write
%store -r run
%store -r plot
%store -r plot_show
%store -r plot_save

# Retrieve model parameters
%store -r length_units
%store -r time_units
%store -r nper
%store -r cell_size_x
%store -r cell_size_y
%store -r gw_mod_depth
%store -r z
%store -r kh
%store -r kv
%store -r gw_offset
%store -r porosity
%store -r rch_iface
%store -r rch_iflowface
%store -r recharge_rate
%store -r nstp
%store -r perlen
%store -r tsmult

# Retrieve spatial data
%store -r hec_ras_crs
%store -r terrain_elevation
%store -r raster_transform
%store -r transform
%store -r raster_crs
%store -r output_raster
%store -r cropped_output_raster
%store -r ground_water_domain
%store -r left_boundary
%store -r right_boundary

# retrieve model domain data
%store -r terrain_elevation
%store -r raster_transform
%store -r raster_crs
%store -r raster_bounds_box
%store -r bed_elevation
%store -r raster_width
%store -r raster_height
%store -r ncol
%store -r nrow
%store -r top
%store -r nlay
%store -r grid_x
%store -r grid_y
%store -r grid_points
%store -r intersecting_points
%store -r xorigin
%store -r yorigin
%store -r xmin
%store -r ymin
%store -r xmax
%store -r ymax
%store -r tops
%store -r botm

# retrieve model boundary data
%store -r left_start
%store -r left_end
%store -r right_start
%store -r right_end
%store -r upstream_start_x
%store -r upstream_start_y
%store -r upstream_end_x
%store -r upstream_end_y
%store -r downstream_start_x
%store -r downstream_start_y
%store -r downstream_end_x
%store -r downstream_end_y
%store -r upstream_line
%store -r downstream_line
%store -r upstream_boundary
%store -r downstream_boundary
%store -r grid_cells
%store -r grid_gdf
%store -r idomain

# retrieve model boundary conditions
%store -r boundary_cells
%store -r left_boundary_cells
%store -r right_boundary_cells
%store -r upstream_boundary_cells
%store -r downstream_boundary_cells
%store -r all_boundary_cells
%store -r unique_boundary_cells
%store -r left_boundary_cells_first_layer
%store -r right_boundary_cells_first_layer
%store -r upstream_boundary_cells_first_layer
%store -r downstream_boundary_cells_first_layer
%store -r max_elevation_upstream
%store -r max_elevation_downstream
%store -r gw_elevation_left_first
%store -r gw_elevation_left_last
%store -r gw_elevation_right_first
%store -r gw_elevation_right_last
%store -r gw_elevation_upstream_first
%store -r gw_elevation_upstream_last
%store -r gw_elevation_downstream_first
%store -r gw_elevation_downstream_last
%store -r gw_elevation_left
%store -r gw_elevation_right
%store -r gw_elevation_upstream
%store -r gw_elevation_downstream
%store -r grid_points_coords
%store -r elevation_values
%store -r grid_points_df
%store -r output_csv
%store -r cropped_df
%store -r river_cells
%store -r river_x
%store -r river_y
%store -r river_elevation
%store -r chd_data
%store -r unique_chd_cells
%store -r duplicate_chd_cells
%store -r chd_data_converted

In [2]:
#---------------------- Define Any Wells ------------------------#
# Get the bounds of the shapefile
minx, miny, maxx, maxy = ground_water_domain.total_bounds

# Function to generate a random point within the bounds
def generate_random_point_within(bounds):
    minx, miny, maxx, maxy = bounds
    while True:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if ground_water_domain.contains(pnt).any():
            return pnt

# Generate 3 random points within the shapefile
random_points = [generate_random_point_within((minx, miny, maxx, maxy)) for _ in range(3)]

# Assign pseudo pumping rates to these points
pumping_rates = np.random.uniform(-500, -100, 3)  # Example pseudo pumping rates

# Create a GeoDataFrame with the well locations and pumping rates
well_data = [{'geometry': point, 'pump_rate': rate} for point, rate in zip(random_points, pumping_rates)]
gdf = gpd.GeoDataFrame(well_data)

# Print the GeoDataFrame
print(gdf)

# Save the GeoDataFrame as a shapefile
output_shapefile = 'random_wells.shp'
gdf.to_file(output_shapefile)

print(f"Random wells shapefile saved as {output_shapefile}")

## Read in Actual Point Shapefile (coordinates of wells + well pumping rates)
# Define the path to your point file
## Read in Actual Point Shapefile (coordinates of wells + well pumping rates)
# Define the path to your point file
point_file_path = 'random_wells.shp'  # Replace with the actual path to your point file

# Read the point file using geopandas
gdf = gpd.read_file(point_file_path)

# Reproject the point file to the target CRS
gdf = gdf.to_crs(hec_ras_crs)

# Print the GeoDataFrame
print(gdf)

# Extract well locations and pumping rates
well_locations = gdf['geometry']
pumping_rates = gdf['pump_rate']

# Print well locations and pumping rates
print("Well Locations:")
print(well_locations)
print("Pumping Rates:")
print(pumping_rates)

# Print well locations and pumping rates
print("Well Locations:")
print(well_locations)
print("Pumping Rates:")
print(pumping_rates)

# Define the transform for converting coordinates to model grid indices
# Replace with your actual transform
transform = rasterio.transform.from_origin(xorigin, yorigin, cell_size_x, cell_size_y)

# Create the well package data
wel_data = []
nrow, ncol = idomain.shape[1], idomain.shape[2]

for idx, row in gdf.iterrows():
    well_geometry = row.geometry
    x, y = well_geometry.x, well_geometry.y
    i, j = ~transform * (x, y)  # Convert coordinates to model grid indices
    i, j = abs(int(i)), abs(int(j))  # Ensure indices are integers
    k = 0  # Assuming wells are in the first layer
    q = row['pump_rate']  # Pull the pumping rate from the point file attribute

    # Check if the well point intersects any cell in the grid
    for grid_row in range(nrow):
        for grid_col in range(ncol):
            cell_geometry = grid_gdf.iloc[grid_row * ncol + grid_col].geometry
            if well_geometry.intersects(cell_geometry):
                wel_data.append([k, grid_row, grid_col, float(q)])
                break  # Exit the loop once the intersecting cell is found

# Print the well package data for verification
for entry in wel_data:
    print(entry)

# Monitoring strategy
def monitor_wells(wel_data):
    # Example monitoring strategy: Print well locations and pumping rates
    for well in wel_data:
        k, i, j, q = well
        print(f"Monitoring Well at Layer {k}, Row {i}, Column {j} with Pumping Rate {q} cubic feet per day")

# Implement monitoring strategy
monitor_wells(wel_data)

                           geometry   pump_rate
0   POINT (2407542.737 10515193.36) -429.239722
1  POINT (2407511.264 10515167.775) -118.939439
2  POINT (2407076.271 10515526.465) -331.008306


c:\Users\u4eeevmq\Documents\Python\HyporheicFloPy\.venv\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
INFO:pyogrio._io:Created 3 records


Random wells shapefile saved as random_wells.shp
    pump_rate                          geometry
0 -429.239722   POINT (2407542.737 10515193.36)
1 -118.939439  POINT (2407511.264 10515167.775)
2 -331.008306  POINT (2407076.271 10515526.465)
Well Locations:
0     POINT (2407542.737 10515193.36)
1    POINT (2407511.264 10515167.775)
2    POINT (2407076.271 10515526.465)
Name: geometry, dtype: geometry
Pumping Rates:
0   -429.239722
1   -118.939439
2   -331.008306
Name: pump_rate, dtype: float64
Well Locations:
0     POINT (2407542.737 10515193.36)
1    POINT (2407511.264 10515167.775)
2    POINT (2407076.271 10515526.465)
Name: geometry, dtype: geometry
Pumping Rates:
0   -429.239722
1   -118.939439
2   -331.008306
Name: pump_rate, dtype: float64


[0, 71, 144, -429.2397219198432]
[0, 69, 141, -118.93943880148043]
[0, 104, 97, -331.00830599721064]
Monitoring Well at Layer 0, Row 71, Column 144 with Pumping Rate -429.2397219198432 cubic feet per day
Monitoring Well at Layer 0, Row 69, Column 141 with Pumping Rate -118.93943880148043 cubic feet per day
Monitoring Well at Layer 0, Row 104, Column 97 with Pumping Rate -331.00830599721064 cubic feet per day


In [3]:
# Store new variables
%store wel_data

Stored 'wel_data' (list)
